# House Price Prediction Kaggle Contest

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb

In [0]:
df_train = pd.read_csv('drive/My Drive/Pytorch_DataSet/house-prices/cleaned_train.csv')
df_test = pd.read_csv('drive/My Drive/Pytorch_DataSet/house-prices/cleaned_test.csv')

In [22]:
"""
# For train file

for col in df_train.columns:
  print(df_train[col].isnull().sum())

# For test file

for col in df_test.columns:
  print(df_test[col].isnull().sum())

"""

'\n# For train file\n\nfor col in df_train.columns:\n  print(df_train[col].isnull().sum())\n\n# For test file\n\nfor col in df_test.columns:\n  print(df_test[col].isnull().sum())\n\n'

In [0]:
cat_cols = []

for col in df_train.columns:
  if df_train[col].dtype == 'O':
    cat_cols.append(col)

In [25]:
cat_cols

['MSZoning',
 'LotShape',
 'LotConfig',
 'Neighborhood',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'Foundation',
 'BsmtQual',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'HeatingQC',
 'KitchenQual',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'SaleCondition']

In [0]:
# For train file

for col in df_train.columns:
  if df_train[col].dtype == 'O':
    df_train[col] = df_train[col].astype('category')

# For test file

for col in df_test.columns:
  if df_test[col].dtype == 'O':
    df_test[col] = df_test[col].astype('category')


In [0]:
# df_train.info()

In [0]:
# df_test.info()

In [14]:
"""
print(len(df_test['GarageType'].cat.categories))
print(df_test['GarageType'].cat.categories)
print(df_test['GarageType'].dtype)
"""

6
Index(['2Types', 'Attchd', 'Basment', 'BuiltIn', 'CarPort', 'Detchd'], dtype='object')
category


In [32]:
# Checking different categories for category columns
cols_to_be_removed = []

for col in cat_cols:
  print(col, len(df_train[col].cat.categories), len(df_test[col].cat.categories))

  if len(df_train[col].cat.categories) != len(df_test[col].cat.categories):
    cols_to_be_removed.append(col)
    
  # For train columns
  df_train[col] = df_train[col].cat.codes.values

  # For test columns
  df_test[col] = df_test[col].cat.codes.values



MSZoning 5 5
LotShape 4 4
LotConfig 5 5
Neighborhood 25 25
BldgType 5 5
HouseStyle 8 7
RoofStyle 6 6
Exterior1st 15 13
Exterior2nd 16 15
MasVnrType 4 4
ExterQual 4 4
Foundation 6 6
BsmtQual 4 4
BsmtExposure 4 4
BsmtFinType1 6 6
BsmtFinType2 6 6
HeatingQC 5 5
KitchenQual 4 4
FireplaceQu 6 6
GarageType 6 6
GarageFinish 3 3
SaleCondition 6 6


In [0]:
"""
Now removing those columns from categorical columns whose length of
categories dont match with train and test columns.
As, those columns can't be used for one hot encoding because then dimensions will be different.
"""

for col in cols_to_be_removed:
  cat_cols.remove(col)

In [34]:
df_train.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'LotShape',
       'LotConfig', 'Neighborhood', 'BldgType', 'HouseStyle', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'Foundation',
       'BsmtQual', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtUnfSF', 'TotalBsmtSF', 'HeatingQC', '1stFlrSF',
       '2ndFlrSF', 'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Fireplaces',
       'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish',
       'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', 'MoSold', 'YrSold', 'SaleCondition', 'SalePrice'],
      dtype='object')

In [35]:
cat_cols

['MSZoning',
 'LotShape',
 'LotConfig',
 'Neighborhood',
 'BldgType',
 'RoofStyle',
 'MasVnrType',
 'ExterQual',
 'Foundation',
 'BsmtQual',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'HeatingQC',
 'KitchenQual',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'SaleCondition']

## Splitting into features and labels

In [0]:
# For training 

X = df_train[['MSSubClass','LotFrontage', 'LotArea', 'HouseStyle', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'Exterior1st',
       'Exterior2nd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
       '2ndFlrSF', 'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'TotRmsAbvGrd', 'Fireplaces','GarageYrBlt',
       'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', 'MoSold', 'YrSold']].values

X_cat = df_train[['MSZoning','LotShape','LotConfig','Neighborhood','BldgType','RoofStyle',
 'MasVnrType','ExterQual','Foundation','BsmtQual','BsmtExposure','BsmtFinType1',
 'BsmtFinType2','HeatingQC','KitchenQual','FireplaceQu','GarageType','GarageFinish','SaleCondition']]  


y = df_train['SalePrice'].values


In [0]:
# For test

X_test = df_test[['MSSubClass','LotFrontage', 'LotArea', 'HouseStyle', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'Exterior1st',
       'Exterior2nd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
       '2ndFlrSF', 'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'TotRmsAbvGrd', 'Fireplaces','GarageYrBlt',
       'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', 'MoSold', 'YrSold']].values

X_test_cat = df_test[['MSZoning','LotShape','LotConfig','Neighborhood','BldgType','RoofStyle',
 'MasVnrType','ExterQual','Foundation','BsmtQual','BsmtExposure','BsmtFinType1',
 'BsmtFinType2','HeatingQC','KitchenQual','FireplaceQu','GarageType','GarageFinish','SaleCondition']]  


In [39]:
# Applying One Hot Encoder

from sklearn.preprocessing import OneHotEncoder 
onehotencoder = OneHotEncoder() 

# For Training

X_cat_data = onehotencoder.fit_transform(X_cat).toarray()
print("Training")
print(X_cat_data.shape, type(X_cat_data))
print(X_cat_data)

# For testing

X_test_cat_data = onehotencoder.transform(X_test_cat).toarray()
print("Training")
print(X_test_cat_data.shape, type(X_test_cat_data))
print(X_test_cat_data)


Training
(1460, 114) <class 'numpy.ndarray'>
[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]]
Training
(1459, 114) <class 'numpy.ndarray'>
[[0. 0. 1. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


In [41]:
# Merging the encoded values with real X

# For train
X = np.concatenate((X,X_cat_data),axis=1)
print(X.shape)

# For test

X_test = np.concatenate((X_test,X_test_cat_data),axis=1)
print(X_test.shape)

(1460, 259)
(1459, 259)


In [43]:
input_data = xgb.DMatrix(data=X,label=y)
input_data